# Study Summary

1. 三個方法，大幅減少程式的執行時間
2. 比較使用內建函數，和自訂函數的程式讀取

1. 讀取資料型態最快速的 (讀取速度 pkl > hdf > csv > xlsx)
   pkl: Python中存放文件的格式，以序列化方式存放
   hdf: 儲存組織大量資料的格式
   csv: 以逗號區隔開來的格式
   xlsx: 存儲excel的格式
2. 多使用內建函數，而非自訂函數
3. 向量化的資料處理 (把一筆一筆的資料轉為陣列dataframe，也就是numpy數值計算)

2. 比較使用內建函數，和自訂函數的程式讀取速度

In [1]:
import pandas as pd
import numpy as np
import time

score_df = pd.DataFrame([[1, 50, 80, 70, 'boy', 1],
                         [2, 60, 45, 70, 'boy', 1],
                         [3, 98, 43, 70, 'boy', 1],
                         [4, 70, 69, 70, 'boy', 1],
                         [5, 56, 79, 70, 'boy', 1],
                         [6, 60, 68, 70, 'girl', 2],
                         [7, 45, 70, 70, 'girl', 2],
                         [8, 55, 77, 70, 'girl', 2],
                         [9, 25, 57, 70, 'girl', 2],
                         [10, 88, 40, 70, 'girl', 2],
                         [11, 25, 60, 70, 'girl', 2],
                         [12, 80, 90, 70, 'girl', 2],
                         [13, 20, 50, 70, 'boy', 1],
                         [14, 50, 67, 70, 'boy', 1],
                         [15, 89, 50, 70, 'boy', 1]],
                         columns = ['student_id', 'math_score', 'english_score', 'chinese_socre', 'gender', 'class'])
score_df

,student_id,math_score,english_score,chinese_socre,gender,class
0,1,50,80,70,boy,1
1,2,60,45,70,boy,1
2,3,98,43,70,boy,1
3,4,70,69,70,boy,1
4,5,56,79,70,boy,1
5,6,60,68,70,girl,2
6,7,45,70,70,girl,2
7,8,55,77,70,girl,2
8,9,25,57,70,girl,2
9,10,88,40,70,girl,2


In [6]:
#python內建函式
star_time = time.time()
score_df.groupby('class').agg('mean')
end_time = time.time()
end_time - star_time

0.0019986629486083984

In [7]:
#python自訂語法
star_time = time.time()
score_df.groupby('class').agg(lambda x:x.mean())
end_time = time.time()
end_time - star_time

C:\Users\user\anaconda3\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


0.019887685775756836

In [8]:
#python的內建函式transform+內建mean
star_time = time.time()
score_df.groupby('class').transform('mean')
end_time = time.time()
end_time - star_time

C:\Users\user\AppData\Local\Temp/ipykernel_11068/3573211492.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  score_df.groupby('class').transform('mean')


0.012996196746826172

In [9]:
#python的內建函式transform+自訂
star_time = time.time()
score_df.groupby('class').transform(lambda x:x.mean())
end_time = time.time()
end_time - star_time

C:\Users\user\AppData\Local\Temp/ipykernel_11068/3778532194.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  score_df.groupby('class').transform(lambda x:x.mean())


0.010998725891113281

結論: python內建函式的讀取速度最快，耗用的時間最低

大型資料集處理-欄位型態降級

1. 建立大資料集(浮點數、整數資料集)

In [9]:
float_data = pd.DataFrame(np.random.uniform(0,5,100000).reshape(1000,100))
int_data = pd.DataFrame(np.random.randint(0,1000,100000).reshape(1000,100))
int_data.memory_usage(deep=True).sum(), float_data.memory_usage(deep=True).sum()

(400128, 800128)

2. 整數型態int改成unit減少記憶體使用空間
   int_data.apply

In [10]:
downcast_int = int_data.apply(pd.to_numeric, downcast = 'unsigned')
int_data.memory_usage(deep = True).sum(), downcast_int.memory_usage(deep=True).sum()

(400128, 200128)

In [18]:
#驗證資料都從int32 downcast uint16
compare_int = pd.concat([int_data.dtypes, downcast_int.dtypes], axis = 1)
compare_int.columns = ['before', 'after']
compare_int.apply(pd.value_counts)

#補充: value_counts用法可參考: https://communeit.medium.com/9%E5%80%8Bvalue-counts-%E7%9A%84%E5%B0%8F%E6%8A%80%E5%B7%A7-%E6%8F%90%E9%AB%98pandas-%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90%E6%95%88%E7%8E%87-5af966ac5708

,before,after
uint16,NaN,100.0
int32,100.0,NaN


3. 將浮點數型態 float64 改成 float32 
   float_data.apply

In [19]:
downcast_float = float_data.apply(pd.to_numeric, downcast = 'float')
float_data.memory_usage(deep = True).sum(), downcast_float.memory_usage(deep = True).sum()

(800128, 400128)

In [21]:
#驗證資料都從float64 downcast float32
compare_int = pd.concat([float_data.dtypes, downcast_float.dtypes], axis = 1)
compare_int.columns = ['before', 'after']
compare_int.apply(pd.value_counts)

,before,after
float32,NaN,100.0
float64,100.0,NaN


# Homework

1. 在速度較慢的時候，可以先從哪邊開始檢查？
2. 資料過大時應採取什麼方式讓記憶體占用量下降？

1. 在速度較慢的時候，可以先從哪邊開始檢查？
   先檢查是否有自訂義函數，有的話，檢查Pandas是否有支援相關演算法，若無法改善，則從資料讀取方式下手，
   將讀取的檔案改為pkl檔

2. 資料過大時應採取什麼方式讓記憶體占用量下降？
   將欄位的型態降級